In [ ]:
import json
import pandas as pd
from lib.utils import BASE_DIR,SHEETS_DIR,OUTPUT_DIR

from lib.dataframe_to_text import dataframe_to_text



def dataframe_to_tree(data:pd.DataFrame):
    def col_to(data:pd.DataFrame,index:int):
        return list(data.columns.values[0:index])

    def accum_col(data:pd.DataFrame):
        col_1=data.columns.values[-1]
        col_2=data.columns.values[-2]
        data=data\
            .groupby(col_to(data,-1), as_index=False,sort=False)\
            .agg(lambda x:list(x))
        data[col_2]=data.loc[:,[col_2,col_1]].apply(lambda x:{col_2:x[col_2],"list":x[col_1]},axis=1)
        return data.drop(col_1,axis=1)

    for _ in range(1,len(data.columns)):
        data=accum_col(data)

    return list(data.iloc[:,0])

r4=pd.read_csv(f"{OUTPUT_DIR}/outcomes.csv")
r4_l1=pd.read_csv(f"{OUTPUT_DIR}/outcomes_l1.csv")
r4_l2=pd.read_csv(f"{OUTPUT_DIR}/outcomes_l2.csv")
r4=pd.merge(r4_l2,r4,how="right",on="第2層")
r4=pd.merge(r4_l1,r4,how="right",on="第1層")
r4=r4.dropna(subset=["第1層","第2層","第3層","第4層"]).fillna("")
tree = dataframe_to_tree(r4.loc[:,["第1層","第2層","第3層","第4層"]])
with open(f"{OUTPUT_DIR}/outcomes.json","w") as f:
    json.dump(tree,f,ensure_ascii=False)
